In [ ]:
import dis
import pandas as pd
import yfinance as yf
import numpy as np

from matplotlib.pyplot import new_figure_manager
from requests import get
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_rows', None)  # 모든 행 출
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.expand_frame_repr', None)  # 긴 데이터 프레임 줄바꿈 없이 출력

# tickers=['VTI','TLT','BIL','GLD']
# tickers=['VTI', 'TLT','IEF','GSG','GLD']
# tickers =['SPY','IWM', 'MTUM', 'EFA']
# tickers = ['SPY','IWD','IWM', 'IWN','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','BWX','GLD']
# tickers = ['SPY', 'IWM', 'MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','GLD']
tickers = ['TLT', 'IEF', 'LQD', 'DBC']
start_date = '2016-01-01'
end_date = '2016-12-31'
months = [1]

#  배당데이터 가져오기
def get_dividends(tickers:list):
    ticker_dfs = {}

    for ticker in tickers:
        info = yf.Ticker(ticker)
        df = info.actions.reset_index()
        df['Date'] = df['Date'].dt.date # 년월일만 남김
        df['Symbol'] = ticker
        df = df[['Date','Symbol','Dividends']]
        ticker_dfs[ticker]=df

    return ticker_dfs


# 주가 데이터 가져오기
def get_stock_data(tickers:list, start, end):
    df = yf.download(tickers=tickers, start=start, end=end, progress=False, auto_adjust=False)

    dividends_dfs = get_dividends(tickers)

    # 티커별로 데이터프레임 분리하여 저장할 딕셔너리 초기화
    ticker_dfs = {}

    # 각 티커에 대한 데이터프레임을 생성
    for ticker in tickers:
        # 특정 티커에 대한 데이터를 선택
        df_ticker = df.loc[:, df.columns.get_level_values(1)==ticker]

        # MultiIndex 컬럼에서 티커 레벨 제거
        df_ticker.columns = df_ticker.columns.droplevel(1)
        df_ticker = df_ticker.reset_index()
        df_ticker .columns.name = None

        # 심볼 추가
        df_ticker['Symbol'] = ticker
        
        # Columns 위치 수정
        columns = ['Date', 'Symbol', 'Open', 'Close', 'Adj Close', 'High', 'Low', 'Volume']
        df_ticker = df_ticker[columns]
        df_dividend = dividends_dfs[ticker]

        # DataFrame의 'Date' 컬럼을 datetime 타입으로 변환
        df_dividend['Date'] = pd.to_datetime(df_dividend['Date'])
        df_ticker['Date'] = pd.to_datetime(df_ticker['Date'])

        # 'Date'와 'Symbol'을 기준으로 df_ticker에 df_dividend의 Dividends 컬럼 추가
        df_ticker = df_ticker.merge(df_dividend[['Date', 'Symbol', 'Dividends']], on=['Date', 'Symbol'], how='left')
        
        # 딕셔너리에 저장
        ticker_dfs[ticker] = df_ticker
    
    return ticker_dfs


# 그 기간동안 월말 데이터만 가져오기
def get_stock_data_last_month(tickers:list, start, end):
    ticker_dfs = get_stock_data(tickers, start, end)
    
    for ticker, df in ticker_dfs.items():
        df_new = df.copy()
        df_new = df_new.set_index('Date') # 인덱스를 날짜로 변경
        df_new = df_new.resample('ME').last()
        df_new = df_new.reset_index()
        ticker_dfs[ticker] = df_new

    return ticker_dfs


# 이동 평균 데이터 가져오기(일별 이동평균)
def get_stock_data_with_moving_avg(tickers:list, start:str, end, mads=[10,20,60,200], mams=[10], key = 'Close'):
    sort_mad = sorted(mads)
    last_mad = sort_mad[-1]
    sort_mam = sorted(mams)
    last_mam = sort_mam[-1]*25
    max_val = max(last_mad, last_mam)

    if isinstance(start, str):
        start_date = datetime.strptime(start, "%Y-%m-%d")
    else:
        start_date = start
        
    
    prev_date = start_date - relativedelta(days = max_val*1.5)
    ticker_dfs = get_stock_data(tickers, prev_date, end)
    
    for ticker, df in ticker_dfs.items():
        df = df.copy()

        # 일별 이동평균 구하기
        for mad in mads:
            df[f'SMA_{mad}'] = df[key].rolling(window=mad).mean()

        # ✅ 2. 월말 종가 기준으로 월별 이동평균 추가
        # df = df.set_index('Date')
        monthly_df = df.groupby(df['Date'].dt.to_period('M')).apply(lambda x: x.iloc[-1]).reset_index(drop=True)
        monthly_df = monthly_df[['Date', key]]

        for mam in mams:
            monthly_df[f'MMA_{mam}'] = monthly_df[key].rolling(window=mam).mean()

        # ✅ 3. 다시 일별 데이터와 merge (forward fill)
        monthly_df = monthly_df.drop(columns=[key])  # MMA만 남기고
        df = df.merge(monthly_df, left_on='Date', right_on='Date', how='left')
        # df.fillna(method='ffill', inplace=True) > 이거 추가되면 Nan으로 비어있는 부분에도 같은 값이 다 채워짐.

        df = df[(df['Date'] >= start) & (df['Date']<= end)]
        ticker_dfs[ticker] = df


    return ticker_dfs



# 주가정보와 함께 이동평균선도 같이 들고오기
def get_stock_data_last_month_with_moving_avg(tickers:list, start, end, mads=[200], mams=[10], key = 'Close'):
    ticker_dfs = get_stock_data_with_moving_avg(tickers, start, end, mads, mams, key)
    for ticker, df in ticker_dfs.items():
        df_new = df.copy()
        df_new = df_new.set_index('Date') # 인덱스를 날짜로 변경
        df_new = df_new.resample('ME').last()
        df_new = df_new.reset_index()
        ticker_dfs[ticker] = df_new

    return ticker_dfs


# N개월 수익률 비교하기
def get_return_ratio_month(tickers:list, start, end, months:list = [1,3,6,12], key = 'Close'):
    sort_months = sorted(months)
    last_month = sort_months[-1]

    start_date = datetime.strptime(start, "%Y-%m-%d")
    prev_date = start_date - relativedelta(months=last_month+1) # 
    result_df = {}

    ticker_dfs = get_stock_data_last_month_with_moving_avg(tickers=tickers, start=prev_date, end=end, key=key)
    for ticker, df in ticker_dfs.items():
        month_dfs = {}
        for month in sort_months:
            new_df = df.copy()
            new_df = new_df.fillna(0)
            new_df = new_df.sort_values(by='Date').reset_index(drop=True)
            shift_df = new_df.shift(month)
            new_df['Compare Month'] = month
            new_df['Return Value'] = new_df[key] - shift_df[key]
            new_df['Return Ratio'] = ((new_df['Return Value']/shift_df[key])*100).round(2)
            month_dfs[month]=new_df

        ratio_df = []
        for rows in zip(*(month_df.iterrows() for _, month_df in month_dfs.items())):
            avg_ratio = sum(row['Return Ratio'] for _, row in rows)/len(rows)

            # 예기서 avg_ratio 가 Nan인 값들은 제외시킨다
            if pd.isna(avg_ratio):
                continue

            ratio_df.append((rows[0][1]['Date'], months, avg_ratio))


        df_columns =  df.columns
        avg_columns = ['Date', 'Avg Months', 'Avg Ratio']
        avg_df = pd.DataFrame(ratio_df, columns=avg_columns)

        init_balance = 10000

        # 초기 벨런스 설정
        avg_df['Balance'] = float(init_balance)
        for i in range(1, len(avg_df)):
            avg_df.loc[i, 'Balance'] = avg_df.loc[i - 1, 'Balance'] * (1 + avg_df.loc[i, 'Avg Ratio'] / 100)

        avg_columns.append('Balance')
        avg_df = pd.merge(avg_df, df, on='Date', how='left')     
        expected_columns = list(df_columns) + [col for col in avg_columns if col != 'Date']
        avg_df = avg_df[expected_columns]
        result_df[ticker]=avg_df

    return result_df


# 퍼포먼스를 구하는 공식
def get_performance(df):
    # 심볼과 기본 데이터 설정
    symbol = df['Symbol'].iloc[0]
    start_balance = df['Balance'].iloc[0]
    end_balance = df['Balance'].iloc[-1]
    start_date = pd.to_datetime(df['Date'].iloc[0])
    end_date = pd.to_datetime(df['Date'].iloc[-1])
    num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

    # 연복리 성장률 (CAGR) 계산
    years = num_months / 12
    CAGR = ((end_balance / start_balance) ** (1 / years)) - 1

    # 월간 수익률 계산
    monthly_returns = df['Balance'].pct_change().dropna()

    # 무위험 수익률 가정 (연 2%를 월간으로 환산)
    risk_free_rate = 0.02 / 12

    # 연간화된 표준 편차 계산
    annualized_std_dev = monthly_returns.std() * np.sqrt(12)

    # Sharpe Ratio 계산
    sharpe_ratio = (monthly_returns.mean() - risk_free_rate) / monthly_returns.std() * np.sqrt(12)

    # 최대 낙폭 (MDD) 계산
    running_max = np.maximum.accumulate(df['Balance'])
    drawdown = (df['Balance'] / running_max - 1).min()

    # 결과 데이터프레임 생성
    results = pd.DataFrame({
        "Symbol": [symbol],
        "End Balance": [end_balance],
        "Annualized Return (CAGR)": [CAGR],
        "Standard Deviation": [annualized_std_dev],
        "Sharpe Ratio": [sharpe_ratio],
        "Maximum Drawdown": [drawdown]
    })

    return results


# 여러개의 토탈 퍼포먼스를 구하는 공식
def get_total_performance(ticker_dfs, ratios):
    # 각 데이터 프레임의 'Date', 'Balance', 'Avg Ratio'를 선택하고 투자 비율에 따라 가중치를 적용
    weighted_dfs = []
    for ticker, df in ticker_dfs.items():
        weighted_df = df[['Date', 'Balance', 'Avg Ratio']].copy()
        idx = list(ticker_dfs.keys()).index(ticker)
        weighted_df['Weighted Balance'] = weighted_df['Balance'] * ratios[idx] / 100
        weighted_df['Weighted Avg Ratio'] = weighted_df['Avg Ratio'] * ratios[idx] / 100
        weighted_dfs.append(weighted_df)
    
    # 모든 데이터 프레임을 하나로 합치기
    combined_df = pd.concat(weighted_dfs)
    
    # Date 별로 그룹화하여 'Weighted Balance'와 'Weighted Avg Ratio'의 합 계산
    result_df = combined_df.groupby('Date').agg({
        'Weighted Balance': 'sum',
        'Weighted Avg Ratio': 'sum'
    }).reset_index()
    
    # 결과 DataFrame에 Symbols와 Asset Ratios 추가
    result_df['Symbol'] = [list(ticker_dfs.keys())] * len(result_df)
    result_df['Asset Ratios'] = [ratios] * len(result_df)
    
    # 컬럼 이름 변경
    result_df.rename(columns={'Weighted Balance': 'Balance', 'Weighted Avg Ratio': 'Avg Ratio'}, inplace=True)
    
    return result_df


# 리벨런싱 코드 (매월, 매달, 쿼터)
def get_total_performance_with_rebalance(ticker_dfs, ratios, rebalance='monthly'):
    import pandas as pd

    tickers = list(ticker_dfs.keys())
    ratios = [r / sum(ratios) for r in ratios]

    # 날짜 정렬
    for df in ticker_dfs.values():
        df.sort_values('Date', inplace=True)

    dates = ticker_dfs[tickers[0]]['Date'].tolist()
    start_date = pd.to_datetime(dates[0])
    rebalance_month = start_date.month

    # 초기 설정
    initial_balance = 10000
    balances = {ticker: initial_balance * ratio for ticker, ratio in zip(tickers, ratios)}
    total_performance = []

    for i in range(len(dates)):
        current_date = pd.to_datetime(dates[i])
        current_prices = {ticker: ticker_dfs[ticker].iloc[i]['Close'] for ticker in tickers}

        if i == 0:
            total_balance = sum(balances[ticker] for ticker in tickers)
            asset_ratios = [round((balances[ticker] / total_balance) * 100, 2) for ticker in tickers]
            row = {
                'Date': dates[i],
                'Balance': total_balance,
                'Avg Ratio': 0.0,
                'Symbol': tickers,
                'Asset Ratios': asset_ratios
            }
            # 각 자산별 잔고도 추가
            for ticker in tickers:
                row[ticker] = balances[ticker]
            total_performance.append(row)
            continue

        # 전월 가격 & 자산가치
        prev_prices = {ticker: ticker_dfs[ticker].iloc[i - 1]['Close'] for ticker in tickers}
        prev_total_balance = sum(balances[ticker] for ticker in tickers)

        # 리벨런싱 여부 판단
        if rebalance == 'monthly':
            rebalance_now = True
        elif rebalance == 'yearly':
            rebalance_now = current_date.month == rebalance_month
        elif rebalance == 'quarterly':
            months_passed = (current_date.year - start_date.year) * 12 + (current_date.month - start_date.month)
            rebalance_now = months_passed % 3 == 0
        else:
            rebalance_now = False

        # 가격 변화 반영
        for ticker in tickers:
            balances[ticker] *= current_prices[ticker] / prev_prices[ticker]

        # 리벨런싱 적용
        if rebalance_now:
            total_balance = sum(balances[ticker] for ticker in tickers)
            balances = {ticker: total_balance * ratio for ticker, ratio in zip(tickers, ratios)}

        # 계산 후 기록
        total_balance = sum(balances[ticker] for ticker in tickers)
        avg_ratio = ((total_balance / prev_total_balance) - 1) * 100
        asset_ratios = [round((balances[ticker] / total_balance) * 100, 2) for ticker in tickers]

        row = {
            'Date': dates[i],
            'Balance': total_balance,
            'Avg Ratio': round(avg_ratio, 4),
            'Symbol': tickers,
            'Asset Ratios': asset_ratios
        }
        # 자산별 잔고도 같이 기록
        for ticker in tickers:
            row[ticker] = balances[ticker]

        total_performance.append(row)

    return pd.DataFrame(total_performance)


# TAA 균등
def get_taa_performance(ticker_dfs):
    import pandas as pd

    tickers = list(ticker_dfs.keys())

    # 날짜 정렬
    for df in ticker_dfs.values():
        df.sort_values('Date', inplace=True)

    dates = ticker_dfs[tickers[0]]['Date'].tolist()
    start_date = pd.to_datetime(dates[0])

    initial_balance = 10000
    balances = {ticker: 0.0 for ticker in tickers}
    cash = initial_balance

    total_performance = []

    for i in range(len(dates)):
        current_date = pd.to_datetime(dates[i])

        # 현재 수익률(Avg Ratio), 종가, 이동평균
        avg_ratios = {}
        current_prices = {}
        sma_200s = {}

        for ticker in tickers:
            row = ticker_dfs[ticker].iloc[i]
            avg_ratios[ticker] = row['Avg Ratio']
            current_prices[ticker] = row['Close']
            sma_200s[ticker] = row['SMA_200']

        # 리밸런싱 시점: 매월 리밸런싱
        if i > 0:
            # 가격 변화 반영
            for ticker in tickers:
                prev_price = ticker_dfs[ticker].iloc[i - 1]['Close']
                balances[ticker] *= current_prices[ticker] / prev_price

        # 총 자산 계산
        total_balance = sum(balances.values()) + cash

        # TAA 전략 적용: 상위 3개 선택
        sorted_tickers = sorted(avg_ratios.items(), key=lambda x: x[1], reverse=True)
        top_assets = [ticker for ticker, _ in sorted_tickers[:3]]

        selected_assets = []
        for ticker in top_assets:
            if current_prices[ticker] >= sma_200s[ticker]:
                selected_assets.append(ticker)

        # 비중 계산
        if selected_assets:
            ratio = 1.0 / len(selected_assets)
        else:
            ratio = 0.0  # 현금만 보유

        # 리밸런싱
        balances = {ticker: 0.0 for ticker in tickers}
        cash = total_balance

        for ticker in selected_assets:
            invested = total_balance * ratio
            balances[ticker] = invested
            cash -= invested

        # 자산 비중 계산
        total_balance = sum(balances.values()) + cash
        asset_ratios = [round((balances[ticker] / total_balance) * 100, 2) for ticker in tickers]
        asset_ratios.append(round((cash / total_balance) * 100, 2))  # 현금 비중도 포함

        row = {
            'Date': dates[i],
            'Balance': total_balance,
            'Asset Ratios': asset_ratios,
            'Symbol': tickers + ['Cash']
        }

        for ticker in tickers:
            row[ticker] = balances[ticker]
        row['Cash'] = cash

        total_performance.append(row)

    return pd.DataFrame(total_performance)


# TAA 고정
def get_taa_performance_hold_cash_fixed_ratio(ticker_dfs):
    import pandas as pd

    tickers = list(ticker_dfs.keys())

    for df in ticker_dfs.values():
        df.sort_values('Date', inplace=True)

    dates = ticker_dfs[tickers[0]]['Date'].tolist()
    start_date = pd.to_datetime(dates[0])

    initial_balance = 10000
    balances = {ticker: 0.0 for ticker in tickers}
    cash = initial_balance

    total_performance = []

    for i in range(len(dates)):
        current_date = pd.to_datetime(dates[i])

        avg_ratios = {}
        current_prices = {}
        sma_200s = {}

        for ticker in tickers:
            row = ticker_dfs[ticker].iloc[i]
            avg_ratios[ticker] = row['Avg Ratio']
            current_prices[ticker] = row['Close']
            sma_200s[ticker] = row['MMA_10']

        # 지난달 자산가치 업데이트
        if i > 0:
            for ticker in tickers:
                prev_price = ticker_dfs[ticker].iloc[i - 1]['Close']
                balances[ticker] *= current_prices[ticker] / prev_price

        total_balance = sum(balances.values()) + cash

        # 상위 3개 자산 추출
        sorted_tickers = sorted(avg_ratios.items(), key=lambda x: x[1], reverse=True)
        top3_assets = [ticker for ticker, _ in sorted_tickers[:3]]

        # 비율 고정: [33.3, 33.3, 33.4]
        fixed_ratios = [33.3, 33.3, 33.4]
        selected_assets = []
        new_balances = balances.copy()
        available_cash = total_balance

        for idx, ticker in enumerate(top3_assets):
            if current_prices[ticker] >= sma_200s[ticker]:
                invest_amount = int(total_balance * (fixed_ratios[idx] / 100))
                new_balances[ticker] = invest_amount
                available_cash -= invest_amount
            else:
                # 조건 만족하지 않으면 투자 안 함 (비율만큼 현금 보유)
                new_balances[ticker] = 0.0

        # 나머지 자산은 그대로 유지 (투자 X)
        for ticker in tickers:
            if ticker not in top3_assets:
                new_balances[ticker] = 0.0

        cash = available_cash
        balances = new_balances

        # 총 잔고 계산
        total_balance = sum(balances.values()) + cash
        asset_ratios = [round((balances[ticker] / total_balance) * 100, 2) for ticker in tickers]
        asset_ratios.append(round((cash / total_balance) * 100, 2))

        row = {
            'Date': dates[i],
            'Balance': total_balance,
            'Asset Ratios': asset_ratios,
            'Symbol': tickers + ['Cash']
        }

        for ticker in tickers:
            row[ticker] = balances[ticker]
        
        row['Cash'] = cash

        if i > 0:
            prev_balance = total_performance[i-1]['Balance']
            row['Ratio'] = round(((total_balance - prev_balance) / prev_balance) * 100, 2)
        total_performance.append(row)

    return pd.DataFrame(total_performance)


# ticker_dfs = 
# display(ticker_dfs['SPY'])
# df = get_taa_performance(ticker_dfs)
# df = get_taa_performance_hold_cash_fixed_ratio(ticker_dfs)
# display(df)

# df = get_performance(df)
# display(df)
df = get_stock_data_last_month_with_moving_avg(tickers, start_date, end_date)
df = get_return_ratio_month(tickers, start_date, end_date)
# for key, value in df.items():
#     display(value)


df = get_taa_performance_hold_cash_fixed_ratio(df)
# df = get_taa_performance(df)
display(df)
display(get_performance(df))


10
10


,Date,Balance,Asset Ratios,Symbol,TLT,IEF,LQD,DBC,Cash,Ratio
0,2015-12-31,10000.000000,"[0.0, 0.0, 0.0, 0.0, 100.0]","[TLT, IEF, LQD, DBC, Cash]",0.0,0.0,0.0,0.0,10000.000000,NaN
1,2016-01-31,10000.000000,"[33.29, 33.29, 0.0, 0.0, 33.42]","[TLT, IEF, LQD, DBC, Cash]",3329.0,3329.0,0.0,0.0,3342.000000,0.00
2,2016-02-29,10140.780135,"[33.29, 33.29, 0.0, 0.0, 33.42]","[TLT, IEF, LQD, DBC, Cash]",3376.0,3376.0,0.0,0.0,3388.780135,1.41
3,2016-03-31,10124.220915,"[33.3, 33.3, 33.4, 0.0, 0.01]","[TLT, IEF, LQD, DBC, Cash]",3371.0,3371.0,3381.0,0.0,1.220915,-0.16
4,2016-04-30,10125.054721,"[33.29, 33.39, 33.29, 0.0, 0.02]","[TLT, IEF, LQD, DBC, Cash]",3371.0,3381.0,3371.0,0.0,2.054721,0.01
5,2016-05-31,10110.745081,"[33.29, 33.39, 33.29, 0.0, 0.03]","[TLT, IEF, LQD, DBC, Cash]",3366.0,3376.0,3366.0,0.0,2.745081,-0.14
6,2016-06-30,10530.280672,"[33.29, 0.0, 33.4, 33.29, 0.01]","[TLT, IEF, LQD, DBC, Cash]",3506.0,0.0,3517.0,3506.0,1.280672,4.15
7,2016-07-31,10388.847567,"[33.3, 33.39, 33.3, 0.0, 0.02]","[TLT, IEF, LQD, DBC, Cash]",3459.0,3469.0,3459.0,0.0,1.847567,-1.34
8,2016-08-31,10304.878643,"[33.29, 33.39, 33.29, 0.0, 0.02]","[TLT, IEF, LQD, DBC, Cash]",3431.0,3441.0,3431.0,0.0,1.878643,-0.81
9,2016-09-30,10230.711881,"[33.29, 0.0, 33.4, 33.29, 0.02]","[TLT, IEF, LQD, DBC, Cash]",3406.0,0.0,3417.0,3406.0,1.711881,-0.72


,Symbol,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[TLT, IEF, LQD, DBC, Cash]",10082.102265,0.00821,0.056563,-0.183389,-0.055585
